In [ ]:
import time
import statistics as stats
import multiprocessing
import time 
import random
import time
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
start_program = time.time()
from multiprocessing import *
import pandas as pd
import numpy as np
from threading import Thread
import time
import subprocess 


In [ ]:
results = [[] for i in range(5)]

n_it = 100


n_st = number_stations = 10
n_s = number_sensors = 100

def_st = 1
def_s = 3
def_r = 'day'


set_date = [random.random() for i in range(500)]


In [ ]:

def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))



def random_date(start, end, prop, dform = '%Y-%m-%dT%H:%M:%S'):
    return str_time_prop(start, end, dform, prop)
    
    
def to_pm(v):
    return str(round(v[0][0],2)) + "$" + '\\' + "pm$" + str(round(v[1][0],2))


# Queries

In [ ]:
# q_q1 = """select ts, s<sid> FROM d1 where id_station='st<stid>' AND  ts < '<timestamp>' AND ts >  '<timestamp>' - <nb>*<rangesUnit>* 1000000L """
q_q1 = """
    select ts, id_station, <sid> FROM <db> 
    where id_station in <stid>
    AND  ts < '<timestamp>' AND ts >  '<timestamp>' - <nb>*<rangesUnit>* 1000000L 
"""
q_q2 = """
    select ts, id_station, <sid> FROM <db> 
    where id_station in <stid>
    AND  ts < '<timestamp>' AND ts >  '<timestamp>' - <nb>*<rangesUnit>* 1000000L 
    and <sfilter>;
"""
q_q3 = """
    SELECT id_station, <avg_s> FROM <db> 
    WHERE  ts < '<timestamp>' AND ts >  '<timestamp>' - <nb>*<rangesUnit>* 1000000L 
    AND id_station in <stid>
    GROUP BY id_station;
"""
q_q4 = """
    SELECT id_station, ts, <avg_s> FROM <db> 
    WHERE ts < '<timestamp>' AND ts >  '<timestamp>' - <nb>*<rangesUnit>* 1000000L 
    AND id_station in <stid> 
    SAMPLE BY 1h;
"""
# q_q5 = """SELECT id_station, ts, avg(s<sid>) FROM d1 WHERE ts IN '<timestamp>;<nb><rangesUnit>' SAMPLE BY 5s FILL(LINEAR) GROUP BY id_station,ts ORDER BY id_station, ts;"""
q_q5 = """
    SELECT id_station, ts, <avg_s> FROM <db> 
    WHERE  ts < '<timestamp>' AND ts >  '<timestamp>' - <nb>*<rangesUnit>* 1000000L 
    AND id_station in <stid> 
    SAMPLE BY 5s FILL(LINEAR) 
    GROUP BY ts, id_station 
    ORDER BY ts;
"""



In [ ]:
import psycopg2
from tqdm import tqdm 
import time

class QuestDB:
 
    # A sample method 
    @staticmethod
    def query(query, db, max_d, rangesUnit, n_it, start_date = "2019-04-01 00:00:00", stop_date = "2019-04-30 00:00:00", n_st = 1, n_s = 10):

        connection = psycopg2.connect(user="admin",
                                          password="quest",
                                          host="diufrm146",
                                          port="8812",
                                          database="d1")
        options = {"day" : 60 * 60* 24,
                   "week" : 60 * 60* 24 * 7,
                   "minute" : 60,
                   "hour" : 60 * 60,
                   "second" : 1,
                   "month" : 60 * 60 * 24 * 30,
                   "year" :  60 * 60 * 24 * 30 * 12
        }
        cursor = connection.cursor()
        cursor.execute("select ts, s9 FROM d1 where id_station='st4' AND ts IN '2019-03-23;1d'")
        cursor.fetchall()
        results = [[],[]]
        
        duration = max_d
        
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
#             time.sleep(0.5)
            date = random_date(start_date, stop_date, set_date[(duration*i)%500], dform = '%Y-%m-%d')
            temp = query.replace("<timestamp>", date+'T12:15')
            temp = temp.replace("<nb>", str(duration))
            temp = temp.replace("<rangesUnit>", str(options[rangesUnit]))
            
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
            q = '(' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ', ' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            
            # sensor
            li = ['s' + str(z) for z in random.sample(range(number_sensors), n_s)]
            q = li[0]
            q_filter = "(" + li[0] + ' > 0.95'
            q_avg = 'avg(' + li[0] + ')'
            for j in li[1:]:
                q += ', ' + j
#                 q_filter += ' OR ' + j + ' > 0.95'
                q_avg += ', ' + 'avg(' + j + ')'
            temp = temp.replace("<db>", db)
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<sfilter>", q_filter + ")")
            temp = temp.replace("<avg_s>", q_avg)
            
            start = time.time()
#             print(temp)
            cursor.execute(temp)
            #print(temp, cursor.rowcount)
            #print(len)
            diff = (time.time()-start)*1000
            cursor.fetchall()
#             print(diff)
#                 print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 20 and i > 5: 
                break                 
#         print(temp)
        results[0].append(stats.mean(runtimes))
#             print(runtimes)
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        connection.close()
        return results
    
    @staticmethod
    def insert_file(batch_size,rate, st):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        conn = psycopg2.connect(user="admin",
                                          password="quest",
                                          host="diufrm146",
                                          port="8812",
                                          database="d1_ingest")
        conn.autocommit = True
        cur = conn.cursor()
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
        f = open(file, "r")
        f = f.read()
        f = f.splitlines()[1:]
        for i in range(len(f)):
            f[i] = f[i].split(',')            
            temp = "'" + "-".join(f[i][0].split("/")) + "' , " + "'" + f[i][1] + "' "
            for e in f[i][2:]:
                temp += ", " + str(e)
            f[i] = temp
        queries = []
        for i in range(0, len(f), rate):
            q = "INSERT INTO d1_ingest VALUES (" +  f[i] + ")"
            for i in f[i+1:i+rate]:
                q += ", \n(" + i + ")"
            queries.append(q)
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            cur.execute(queries[a])
#             if a / 1 == 0 : print(queries[a])
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
        
    @staticmethod
    def insert_file_1station(batch_size,rate, st, f):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        conn = psycopg2.connect(user="admin",
                                          password="quest",
                                          host="diufrm146",
                                          port="8812",
                                          database="d1_ingest")
        conn.autocommit = True
        cur = conn.cursor()
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
#         f = open(file, "r")
#         f = f.read()
#         f = f.splitlines()[1:]
#         for i in range(len(f)):
#             f[i] = f[i].split(',')            
#             temp = "'" + "-".join(f[i][0].split("/")) + "' , " + "'" + f[i][1] + "' "
#             for e in f[i][2:]:
#                 temp += ", " + str(e)
#             f[i] = temp
        queries = []
        for i in range(0, len(f), rate):
            q = "INSERT INTO d1_ingest VALUES (" +  f[i] + ")"
            for i in f[i+1:i+rate]:
                q += ", \n(" + i + ")"
            queries.append(q)
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            cur.execute(queries[a])
#             if a / 1 == 0 : print(queries[a])
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
        
    @staticmethod
    def run_all(start, stop, db):
        results[0].append(to_pm(QuestDB.query(q_q1, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[1].append(to_pm(QuestDB.query(q_q2, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[2].append(to_pm(QuestDB.query(q_q3, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[3].append(to_pm(QuestDB.query(q_q4, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[4].append(to_pm(QuestDB.query(q_q5, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
    
    
    
    

In [ ]:
QuestDB.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1")

In [ ]:
data = []
for st in tqdm(range(10)):    
    file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
    f = open(file, "r")
    f = f.read()
    f = f.splitlines()[1:]
    for i in range(len(f)):
        f[i] = f[i].split(',')            
        temp = "'" + "-".join(f[i][0].split("/")) + "' , " + "'" + f[i][1] + "' "
        for e in f[i][2:]:
            temp += ", " + str(e)
        f[i] = temp
#     f = ["\n".join(f[i:i+rate])  for i in tqdm(range(0, len(f), rate)]
    data.append(f)
data2 = []
for i in range(len(data[0])):
    for j in range(10):
        data2.append(data[j][i])
f = data2

In [ ]:
n_threads = 1
stop_insertion = [False for i in range(n_threads)]
rate = 2000

# 20K dp/s

# Launch insertion
for i in range(n_threads):  
    t1 = Thread(target = QuestDB.insert_file_1station, args=(100_000, rate, i, f))
#     t1 = Thread(target = QuestDB.insert_file, args=(100_000, rate, i))
#     t1 = Thread(target = ClickHouse.insert_by_point, args=(rate, i))
    t1.setDaemon(True)
    t1.start()

# insert_new(200, 1)

In [ ]:
QuestDB.insert_file_1station(100_000, rate, i, f)

In [ ]:
time.sleep(3 * 8640 // rate)

In [ ]:
QuestDB.run_all("2019-04-01T00:00:00", "2019-04-30T00:00:00", "d1_ingest")

In [ ]:
QuestDB.run_all("2019-05-01T01:00:00", "2019-05-02T00:00:00", "d1_ingest")

In [ ]:
QuestDB.run_all("2019-04-29T01:00:00", "2019-05-02T00:00:00", "d1_ingest")

In [ ]:
stop_insertion = [True for i in range(n_threads)]


In [ ]:
results 

In [ ]:
runtimes = [[float(results[i][j].split('$')[0]) for j in range(len(results[i]))][:4] for i in range(5)]
variance = [[float(results[i][j].split('$')[2]) for j in range(len(results[i]))][:4] for i in range(5)]

In [ ]:
runtimes = list(map(list, zip(*runtimes)))
variance = list(map(list, zip(*variance)))


In [ ]:
runtimes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.arange(5)
labels = ['query'+str(1+i) for i in range(5)]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0,0,1,1])
for i in range(4):
    ax.bar(X +0.15*i, runtimes[i], yerr = variance[i], width = 0.15)
ax.legend(['offline', 'online old', 'online new', 'online mixed', ])
ax.set_xticklabels(labels)
ax.set_xticks(np.arange(len(labels)))
ax.yaxis.grid(True)
plt.title('ClickHouse offline vs. online, rate: '+ str(rate) + ' x ' + str(n_threads))
plt.tight_layout()

# ax.bar(X+0.25, data[1], color = 'g', width = 0.25)
# ax.bar(X+0.5, data[2], color = 'r', width = 0.25)

In [ ]:
import psycopg2
conn = psycopg2.connect(user="admin",
                                          password="quest",
                                          host="diufrm146",
                                          port="8812",
                                          database="d1")
conn.autocommit = True
cursor = conn.cursor()
cursor.execute("drop table d1_ingest;")
cursor.execute("CREATE TABLE 'd1_ingest' ( ts TIMESTAMP, id_station SYMBOL capacity 256 CACHE index capacity 256, s0 DOUBLE, s1 DOUBLE, s2 DOUBLE, s3 DOUBLE, s4 DOUBLE, s5 DOUBLE, s6 DOUBLE, s7 DOUBLE, s8 DOUBLE, s9 DOUBLE, s10 DOUBLE, s11 DOUBLE, s12 DOUBLE, s13 DOUBLE, s14 DOUBLE, s15 DOUBLE, s16 DOUBLE, s17 DOUBLE, s18 DOUBLE, s19 DOUBLE, s20 DOUBLE, s21 DOUBLE, s22 DOUBLE, s23 DOUBLE, s24 DOUBLE, s25 DOUBLE, s26 DOUBLE, s27 DOUBLE, s28 DOUBLE, s29 DOUBLE, s30 DOUBLE, s31 DOUBLE, s32 DOUBLE, s33 DOUBLE, s34 DOUBLE, s35 DOUBLE, s36 DOUBLE, s37 DOUBLE, s38 DOUBLE, s39 DOUBLE, s40 DOUBLE, s41 DOUBLE, s42 DOUBLE, s43 DOUBLE, s44 DOUBLE, s45 DOUBLE, s46 DOUBLE, s47 DOUBLE, s48 DOUBLE, s49 DOUBLE, s50 DOUBLE, s51 DOUBLE, s52 DOUBLE, s53 DOUBLE, s54 DOUBLE, s55 DOUBLE, s56 DOUBLE, s57 DOUBLE, s58 DOUBLE, s59 DOUBLE, s60 DOUBLE, s61 DOUBLE, s62 DOUBLE, s63 DOUBLE, s64 DOUBLE, s65 DOUBLE, s66 DOUBLE, s67 DOUBLE, s68 DOUBLE, s69 DOUBLE, s70 DOUBLE, s71 DOUBLE, s72 DOUBLE, s73 DOUBLE, s74 DOUBLE, s75 DOUBLE, s76 DOUBLE, s77 DOUBLE, s78 DOUBLE, s79 DOUBLE, s80 DOUBLE, s81 DOUBLE, s82 DOUBLE, s83 DOUBLE, s84 DOUBLE, s85 DOUBLE, s86 DOUBLE, s87 DOUBLE, s88 DOUBLE, s89 DOUBLE, s90 DOUBLE, s91 DOUBLE, s92 DOUBLE, s93 DOUBLE, s94 DOUBLE, s95 DOUBLE, s96 DOUBLE, s97 DOUBLE, s98 DOUBLE, s99 DOUBLE ) timestamp (ts) PARTITION BY DAY;")
cursor.execute("INSERT INTO d1_ingest SELECT cast(ts AS timestamp) ts, cast(id_station AS symbol) id_station, s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99 FROM 'd1';")




In [ ]:
# insert data